# Laborarbeit Künstliche Intelligenz

## Thema :  Evolutionary Computing & Constraint Satisfaction Problems

### Namen der Studierenden: Michael Dehm & Tim Teller

(Hinweis: Es sind Namen anzugeben und keine Matrikelnummern. Matrikelnummern werden ausschließlich bei Klausuren zur Anonymisierung verwendet)

# Installs

In [ ]:
%pip install seaborn matplotlib pandas deap python-constraint ortools

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import numpy as np
from collections import Counter
import random
from copy import deepcopy

# Einlesen der Konfigurations-Datei für Ihre Aufgabe
def load_configuration():
    with open('configuration_003.json', 'r') as file:
        return json.load(file)

configuration = load_configuration()

#### Die Konfiguration betrachten

In [ ]:
def print_configuration(configuration):
    
    print("\nAusführliche Konfiguration:\n")
    
    print("Studiengänge und Kursgruppenanzahlen:")
    for program, groups in configuration['study_programs'].items():
        print(f"- {program}: {groups} Kursgruppen")
    
    print("\nPrüfungswochen und zugeordnete Studiengänge:")
    for week, programs in configuration['exam_weeks'].items():
        print(f"- {week}: {', '.join(programs)}")
    
    print("\nZeitslots mit Beliebtheit:")
    for slot, details in configuration['time_slots'].items():
        print(f"- {slot} ({details['time']}): Beliebtheit {details['popularity']}")
    
    print("\nTage der Woche mit Beliebtheit:")
    for day, details in configuration['days'].items():
        print(f"- {day}: Beliebtheit {details['popularity']}")
    
    print("\nBeliebtheit von Raumzuteilungen:")
    print(f"- Parallel zugeteilte Räume: Beliebtheit {configuration['room_popularity']['parallel']}")
    print(f"- Zeitlich hintereinander zugeteilte Räume: Beliebtheit {configuration['room_popularity']['sequential']}")


In [ ]:
print_configuration(configuration)

Die hier angegebenen Beliebtheitswerte fließen in die Bewertungsfunktion ein, die von Ihnen noch anzupassen ist.

In [ ]:
# Nur als Beispiel und zur Veranschaulichung ... zufällige Zuteilung generieren
import random


def random_assign_timeslots(week_number, configuration):
    study_programs = configuration['exam_weeks'].get(f"Week {week_number}", [])
    
    # Struktur für die Zuteilung: Dict of days containing list of slots each with two rooms
    schedule = {day: {slot: [None, None] for slot in configuration['time_slots']} for day in configuration['days']}
    
    # Erstelle eine Liste aller Zeit-Slot Kombinationen für eine Woche
    time_slot_combinations = [(day, slot) for day in schedule for slot in schedule[day]]
    
    # Verteile die Studiengänge zufällig, bis alle Slots gefüllt sind
    for (day, slot) in time_slot_combinations:
        for i in range(2):  # Zwei Räume pro Slot
            program = random.choice(study_programs)
            schedule[day][slot][i] = program
    
    return schedule

def print_schedule(schedule):
    days = list(schedule.keys())
    slots = list(schedule[days[0]].keys())
    
    # Erste Zeile: Tage der Woche
    day_line = "Zeit / Tage  " + "".join([f"{day:^20}" for day in days])
    
    # Zweite Zeile: Räume unter jedem Tag anzeigen
    room_line = " " * 12 + "".join([f"{'R1':^10}{'R2':^10}" for _ in days])
    
    line_length = len(day_line)
    line = "-" * line_length
    
    print(line)
    print(day_line)
    print(room_line)
    print(line)

    for slot in slots:
        print(f"{slot:^10} ", end="")
        for day in days:
            rooms = schedule[day][slot]
            r1, r2 = rooms[0], rooms[1]
            print(f"{r1:^10}{r2:^10}", end="")
        print()
    print(line)



In [ ]:
# Beispiel für Verwendung

week_number = 2  # Zum Beispiel die erste Prüfungswoche
schedule = random_assign_timeslots(week_number, configuration)
print(schedule)
print_schedule(schedule)

## Aufgabenteil 1: Evolutionary Computing

Entwerfen Sie ein KI Modell auf Basis des Evolutionary Computing und setzen
Sie dieses als Jupyter Notebook um. Insbesondere werden eine geeignete Repräsentation
sowie eine geeignete Fitness-Funktion benötigt.

In [ ]:

from deap import algorithms, base, creator, tools

### 1. Modell anlegen

- Individuenformat festlegen
- Fitnessfunktion festlegen
- Mutation und Crossover geeignet wählen
- ... weitere Parameter

In [ ]:
# Individuum definieren
# Ihr Code 

# print(schedule)
print_schedule(schedule)

def calc_unfairness(scores_by_study_program):
    vals = list(scores_by_study_program.values())
    var = np.var(vals)
    return var

def calc_fittnes(schedule,week_number,configuration):
    day_num = len(configuration['days'])
    room_num = 2
    slot_num = len(configuration['time_slots'])
    total_slots = day_num*room_num*slot_num

    programs_this_week = configuration['exam_weeks'][f'Week {week_number}']
    courses_per_program = configuration['study_programs']

    total_courses = 0
    for study_program in programs_this_week:
        total_courses += courses_per_program[study_program]
    
    proportional_slots = {}
    for study_program in programs_this_week:
        proportional_slots[study_program] = courses_per_program[study_program]/total_courses * total_slots
    
    all_values = []
    for day_slots in schedule.values():
        for slot in day_slots.values():
            all_values.extend(slot)

    count = Counter(all_values)
    actual_slots = dict(count)

    # print(week_number,programs_this_week)
    # print(proportional_slots)
    # print(actual_slots)

    unproportional_slots_penalty = 0
    for study_program in actual_slots:
        unproportional_slots_penalty += np.abs(actual_slots[study_program]-proportional_slots[study_program])*10
    
    parallel_popularity_scores_by_study_program = {}
    consecutive_popularity_scores_by_study_program = {}
    day_popularity_scores_by_study_program = {}
    slot_popularity_scores_by_study_program = {}
    for study_program in programs_this_week:
        parallel_popularity_scores_by_study_program[study_program] = 0
        consecutive_popularity_scores_by_study_program[study_program] = 0
        day_popularity_scores_by_study_program[study_program] = 0
        slot_popularity_scores_by_study_program[study_program] = 0

    big_course_parallel_popularity = configuration['room_popularity']['parallel']
    for day in schedule:
        for slot in schedule[day]:
            parallel_rooms_by_study_program = Counter(schedule[day][slot])
            for study_program in dict(parallel_rooms_by_study_program):
                if courses_per_program[study_program] > 1 and parallel_rooms_by_study_program[study_program] > 1:
                    parallel_popularity_scores_by_study_program[study_program] += big_course_parallel_popularity

    course_consecutive_popularity = configuration['room_popularity']['sequential']
    for day in schedule:
        for i in range(1,len(schedule[day])):
            prev_key = list(schedule[day])[i-1]
            key = list(schedule[day])[i]
            prev_slot = schedule[day][prev_key]
            slot = schedule[day][key]
            for k in range(0,room_num):
                if(slot[k] in prev_slot):
                    consecutive_popularity_scores_by_study_program[slot[k]] += course_consecutive_popularity
    
    for day in schedule:
        for slot in schedule[day]:
            for study_program in schedule[day][slot]:
                day_popularity = configuration['days'][day]['popularity']
                slot_popularity = configuration['time_slots'][slot]['popularity']
                day_popularity_scores_by_study_program[study_program] += day_popularity
                slot_popularity_scores_by_study_program[study_program] += slot_popularity
                
    for study_program in programs_this_week:
        parallel_popularity_scores_by_study_program[study_program] /= courses_per_program[study_program]
        consecutive_popularity_scores_by_study_program[study_program] /= courses_per_program[study_program]
        day_popularity_scores_by_study_program[study_program] /= courses_per_program[study_program]
        slot_popularity_scores_by_study_program[study_program] /= courses_per_program[study_program]

    # delete score for study_programs with only one course, so the unfairnes score is not negativly impacted
    for study_program in programs_this_week:
        if courses_per_program[study_program] <= 1:
            del parallel_popularity_scores_by_study_program[study_program]

    # print(parallel_popularity_scores_by_study_program)
    # print(consecutive_popularity_scores_by_study_program)
    # print(day_popularity_scores_by_study_program)
    # print(slot_popularity_scores_by_study_program)

    number_of_days_by_program = {}
    for program in programs_this_week:
        number_of_days_by_program[program] = 0

    for day in schedule:
        programs_this_day = []
        for slot in schedule[day]:
            for program in schedule[day][slot]:
                programs_this_day.append(program)
        for program in number_of_days_by_program:
            if program in programs_this_day:
                number_of_days_by_program[program] += 1
    
    slot_not_every_day_penalty = day_num*len(programs_this_week)
    for program in number_of_days_by_program:
        slot_not_every_day_penalty -= number_of_days_by_program[program]


    early_late_slots_penalty_factor = 10
    early_late_slots_penalty = 0
    for i in range(1,len(schedule)):
        prev_day = list(schedule)[i-1]
        day = list(schedule)[i]
        last_slot_prey_day = list(schedule[prev_day].values())[-1]
        first_slot_this_day = list(schedule[day].values())[0]
        # print(prev_day,day)
        # print(last_slot_prey_day,first_slot_this_day)
        for program in last_slot_prey_day:
            if program in first_slot_this_day:
                early_late_slots_penalty += early_late_slots_penalty_factor
    # print(early_late_slots_penalty)
    unfairness_slot_not_every_day = calc_unfairness(number_of_days_by_program)
    unfairness_parallel_popularity_scores = calc_unfairness(parallel_popularity_scores_by_study_program)
    unfairness_consecutive_popularity_scores = calc_unfairness(consecutive_popularity_scores_by_study_program)
    unfairness_day_popularity_scores = calc_unfairness(day_popularity_scores_by_study_program)
    unfairness_slot_popularity_scores = calc_unfairness(slot_popularity_scores_by_study_program)

    # print(unfairness_parallel_popularity_scores)
    # print(unfairness_consecutive_popularity_scores)
    # print(unfairness_day_popularity_scores)
    # print(unfairness_slot_popularity_scores)

    # print(number_of_days_by_program)
    # print(unfairness_slot_not_every_day)
    # print(slot_not_every_day_penalty)

    
    parallel_popularity_score_total = sum(list(parallel_popularity_scores_by_study_program.values()))
    consecutive_popularity_score_total = sum(list(consecutive_popularity_scores_by_study_program.values()))
    day_popularity_score_total = sum(list(day_popularity_scores_by_study_program.values()))
    slot_popularity_score_total = sum(list(slot_popularity_scores_by_study_program.values()))

    total_score     =   parallel_popularity_score_total \
                    +   consecutive_popularity_score_total \
                    +   day_popularity_score_total \
                    +   slot_popularity_score_total \
                    -   unfairness_parallel_popularity_scores \
                    -   unfairness_consecutive_popularity_scores \
                    -   unfairness_day_popularity_scores \
                    -   unfairness_slot_popularity_scores \
                    -   unproportional_slots_penalty \
                    -   unfairness_slot_not_every_day \
                    -   slot_not_every_day_penalty \
                    -   early_late_slots_penalty
    # print(total_score)
    return total_score

# print(schedule)


def mutate_schedule(schedule, week_number, configuration, mutation_probability=0.2):
    from copy import deepcopy
    import random

    room_num = 2
    new_schedule = deepcopy(schedule)

    programs_this_week = configuration['exam_weeks'][f'Week {week_number}']

    all_positions = [
        (day, slot, room)
        for day in new_schedule
        for slot in new_schedule[day]
        for room in range(room_num)
    ]

    if random.random() < mutation_probability:
        mutation_type = random.choice(['swap', 'replace'])

        if mutation_type == 'swap':
            pos1, pos2 = random.sample(all_positions, 2)
            d1, s1, r1 = pos1
            d2, s2, r2 = pos2
            new_schedule[d1][s1][r1], new_schedule[d2][s2][r2] = \
                new_schedule[d2][s2][r2], new_schedule[d1][s1][r1]

        elif mutation_type == 'replace':
            d, s, r = random.choice(all_positions)
            new_schedule[d][s][r] = random.choice(programs_this_week)

    return new_schedule

def crossover_schedules(parent1, parent2):
    child1 = deepcopy(parent1)
    child2 = deepcopy(parent2)

    days = list(parent1.keys())
    crossover_day = random.choice(days)

    # Austausch ab crossover_day
    switch = False
    for day in days:
        if day == crossover_day:
            switch = True
        if switch:
            child1[day], child2[day] = deepcopy(parent2[day]), deepcopy(parent1[day])

    return child1, child2







# Evolution

In [ ]:
import random
import matplotlib.pyplot as plt

start_week = 1
end_week = 8
generations_range = range(0, 100)
population_size = 100

populations_by_week = {}
population_scores_by_week = {}
best_scores_by_week = {}
best_schedule_by_week = {}

for week_number in range(start_week, end_week + 1):
    populations_by_week[week_number] = []
    population_scores_by_week[week_number] = []
    best_scores_by_week[week_number] = []
    best_schedule_by_week[week_number] = None

    for _ in range(population_size):
        schedule = random_assign_timeslots(week_number, configuration)
        populations_by_week[week_number].append(schedule)
        population_scores_by_week[week_number].append(calc_fittnes(schedule, week_number, configuration))

for week_number in range(start_week, end_week + 1):
    population = populations_by_week[week_number]
    population_scores = population_scores_by_week[week_number]

    for _ in generations_range:
        sorted_population = [x for _, x in sorted(zip(population_scores, population), key=lambda pair: pair[0], reverse=True)]
        top_half = sorted_population[:population_size // 2]

        new_population = []
        while len(new_population) < population_size // 2:
            parent1, parent2 = random.sample(top_half, 2)
            child1, child2 = crossover_schedules(parent1, parent2)
            new_population.append(child1)
            if len(new_population) < population_size // 2:
                new_population.append(child2)

        population = top_half + new_population
        population_scores = [calc_fittnes(ind, week_number, configuration) for ind in population]

        populations_by_week[week_number] = population
        population_scores_by_week[week_number] = population_scores
        best_scores_by_week[week_number].append(min(population_scores))
        best_schedule_by_week[week_number] = population[population_scores.index(min(population_scores))]

for week_number in range(start_week, end_week + 1):
    plt.scatter(generations_range, best_scores_by_week[week_number])
plt.show()

for week_number in range(start_week, end_week + 1):
    print_schedule(best_schedule_by_week[week_number])

#### Erläuterung / Begründung zur Modellierung von Individuen

<...Ihr Text ...>

In [ ]:
# Fitnessfunktion definieren
# Ihr Code



#### Erläuterung / Begründung zur Fitnessfunktion

<...Ihr Text ...>

In [ ]:

# Modell ersetzen durch eigenes für diese Aufgabe

# Erstelle die Basisobjekte für DEAP

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_int", random.randint, 0, 100)  # Erzeuge zufällige Integer zwischen 1 und 10
toolbox.register("individual", tools.initRepeat, creator.Individual, 
                 toolbox.attr_int, n=4)  # Erzeuge ein Individuum mit 4 Integern
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("evaluate", evaluate) # Ihre zentrale Stelle ...

Ein Beispiel wie ausgewertet werden kann. Die Bewertung einer Verletzung der proportionalen Raumzuteilung ist mit -10 festgesetzt (nicht ändern).

In [ ]:
def count_slots(schedule):
    slot_count = {}
    
    for day, slots in schedule.items():
        for slot, rooms in slots.items():
            for program in rooms:
                if program not in slot_count:
                    slot_count[program] = 0
                slot_count[program] += 1
    
    return slot_count

def calculate_slot_allocation(week_number, configuration):
    study_programs = configuration['exam_weeks'].get(f"Week {week_number}", [])
    total_courses = sum(configuration['study_programs'][program] for program in study_programs)
    
    # Berechne die proportionalen Slots
    slot_allocation = {}
    for program in study_programs:
        num_courses = configuration['study_programs'][program]
        slot_allocation[program] = round((num_courses / total_courses) * 40)
    return slot_allocation
    
def evaluate_schedule(schedule, configuration):
    # Bewertung basierend auf der Popularität der Slots und Tage
    slot_popularity = configuration['time_slots']
    day_popularity = configuration['days']
    
    score = 0
    for day, slots in schedule.items():
        day_score = day_popularity[day]['popularity']
        for slot, rooms in slots.items():
            slot_score = slot_popularity[slot]['popularity']
            # Addiere die Beliebtheitswerte jedes belegten Slots
            if all(rooms):  # Nur wenn beide Slots belegt sind
                score += (day_score + slot_score)
    
    return score
# Zähle die Slots, die den Studiengängen zugewiesen wurden
slot_counts = count_slots(schedule)
print("Slot-Zuweisung pro Studiengang:", slot_counts)

# Berechne die proportionalen Slot-Zuteilungen
slot_allocations = calculate_slot_allocation(week_number, configuration)
print("Proportionale Slot-Zuteilung pro Studiengang:", slot_allocations)

# Bewertungsfunktion
schedule_score = evaluate_schedule(schedule, configuration)
print("Bewertung des Zeitplans:", schedule_score)

def calculate_differences_and_penalty(slot_counts, slot_allocations):
    differences = {}
    total_penalty = 0
    
    for program in slot_allocations:
        allocated_slots = slot_allocations.get(program, 0)
        counted_slots = slot_counts.get(program, 0)
        
        difference = counted_slots - allocated_slots
        differences[program] = difference
        
        # Berechne die Strafe basierend auf der Abweichung
        penalty = abs(difference) * -10
        total_penalty += penalty
    
    return differences, total_penalty
    
# Berechnungen durchführen
slot_counts = count_slots(schedule)
slot_allocations = calculate_slot_allocation(week_number, configuration)

# Abweichungen und Strafen berechnen
differences, total_penalty = calculate_differences_and_penalty(slot_counts, slot_allocations)

# Ausgabe
print("Abweichungen pro Studiengang:", differences)
print("Gesamtstrafen:", total_penalty)
print("Gesamtbewertung:", total_penalty+schedule_score)

In [ ]:
# Population wählen 

### Begründung zum Modell

<...Ihr Text...>

### 2. Test und Bewertung 

Testen Sie die Funktionsweise Ihres Modells und diskutieren Sie die Ergebnisse.

In [ ]:
# Ihr Code



### Diskussion der Ergebnisse 

<...Ihr Text...>

## Aufgabenteil 2: Constraint Satisfaction Problems

Entwerfen Sie ein KI Modell auf Basis von Constraints und setzen Sie dieses als
Jupyter Notebook um. Wählen Sie geeignete Constraints.

In [ ]:
from ortools.sat.python import cp_model
  

### 1. Variablen bestimmen 

... und ihre Wertebereiche!

In [ ]:
def create_variables_for_week(model, configuration, week):
    """
    Erstellt die Variablen für eine einzelne Woche.
    
    Args:
        model: Das OR-Tools CP-Model
        configuration: Dictionary mit der Konfiguration für das Problem
        week: Die zu betrachtende Woche
        
    Returns:
        variables: Dictionary mit den erstellten Variablen
        sg_in_week: Liste der Studiengänge in dieser Woche
    """
    # Daten aus der Konfiguration extrahieren
    days = list(configuration['days'].keys())
    rooms = ['A', 'B']
    slots = [1, 2, 3, 4]

    # Studiengänge in dieser Woche
    sg_in_week = configuration['exam_weeks'][week]

    # Variablen erstellen
    variables = {}
    for day in days:
        for slot_idx in slots:
            for room in rooms:
                var_name = f"slot_{slot_idx}_room_{room}_{day}_{week}"
                # Der Wertebereich ist die Liste der möglichen Studiengänge für diese Woche
                variables[var_name] = model.NewIntVar(
                    0,
                    len(sg_in_week) - 1,
                    var_name
                )

    return variables, sg_in_week


### Begründung zur Wahl der Variablen

Da jeder Zeitslot einer Prüfungswoche von verschiedenen Studiengängen belegt werden kann, ist es sinnvoll, die Slots als Variablen zu modellieren. Der jeweilige Wertebereich dieser Variablen umfasst die Studiengänge, die in der entsprechenden Woche Prüfungen abhalten können.

### 2. Constraints bestimmen

Hier liegt sicher die Hauptaufgabe. Zur Umsetzung als CSP ist ein geeigneter Fairnessrahmen als Grenzwert anzugeben (und als Constraint umzusetzen)

In [ ]:
def add_week_fairness_constraint(model, variables, configuration, sg_in_week, week, fairness_toleranz=0.3):
    """
    Fügt den Fairness-Constraint für eine Woche zum Modell hinzu.
    """
    # Gesamtzahl der Kursgruppen in dieser Woche berechnen
    gesamt_kursgruppen = sum(configuration['study_programs'].get(sg, 0) for sg in sg_in_week)

    # Gesamtzahl der Slots berechnen
    gesamt_slots = len(variables)

    # Für jeden Studiengang in dieser Woche die faire Anzahl an Slots berechnen
    for sg_idx, sg_name in enumerate(sg_in_week):
        # Anzahl der Kursgruppen dieses Studiengangs
        kursgruppen = configuration['study_programs'].get(sg_name, 0)

        # Proportionale Anzahl an Slots berechnen
        faire_slots = (kursgruppen / gesamt_kursgruppen) * gesamt_slots

        # Toleranzbereich definieren
        min_slots = int(faire_slots * (1 - fairness_toleranz))
        max_slots = int(faire_slots * (1 + fairness_toleranz))

        # Zähle, wie viele Slots dem Studiengang zugewiesen wurden
        sg_slots_indicators = []

        for slot_var in variables:
            # Erstelle eine Binärvariable für diesen Slot und Studiengang
            is_sg_assigned = model.NewBoolVar(f"is_sg_{sg_idx}_assigned_to_{slot_var}")

            # Diese Variable ist 1, wenn der Studiengang diesem Slot zugewiesen ist, sonst 0
            model.Add(variables[slot_var] == sg_idx).OnlyEnforceIf(is_sg_assigned)
            model.Add(variables[slot_var] != sg_idx).OnlyEnforceIf(is_sg_assigned.Not())

            sg_slots_indicators.append(is_sg_assigned)

        # Füge den Constraint hinzu, dass die Anzahl der zugewiesenen Slots
        # innerhalb des Toleranzbereichs liegt
        model.Add(sum(sg_slots_indicators) >= min_slots)
        model.Add(sum(sg_slots_indicators) <= max_slots)


def add_week_dominance_constraint(model, variables, configuration, sg_in_week, week, max_slots_per_sg_per_day=4):
    """
    Fügt einen harten Constraint hinzu, der verhindert, dass ein Studiengang zu viele Slots
    an einem Tag in einer Woche belegt.
    """
    # Daten aus der Konfiguration extrahieren
    days = list(configuration['days'].keys())

    # Für jeden Tag und jeden Studiengang
    for day in days:
        for sg_idx, sg_name in enumerate(sg_in_week):
            # Zähle, wie viele Slots diesem Studiengang an diesem Tag zugewiesen wurden
            sg_slots_today = []

            for slot_idx in [1, 2, 3, 4]:
                for room in ['A', 'B']:
                    var_name = f"slot_{slot_idx}_room_{room}_{day}_{week}"
                    if var_name in variables:
                        is_assigned = model.NewBoolVar(f"{var_name}_is_{sg_name}")
                        model.Add(variables[var_name] == sg_idx).OnlyEnforceIf(is_assigned)
                        model.Add(variables[var_name] != sg_idx).OnlyEnforceIf(is_assigned.Not())
                        sg_slots_today.append(is_assigned)

            # Begrenze die Anzahl der Slots, die ein Studiengang an einem Tag belegen darf
            if sg_slots_today:
                model.Add(sum(sg_slots_today) <= max_slots_per_sg_per_day)


def add_previous_assignments_constraints(model, variables, sg_in_week, week, unpopular_terms, unpopular_days, unpopular_slot_indices, configuration):
    # Daten aus der Konfiguration extrahieren
    days = list(configuration['days'].keys())

    # Berechne die relative Größe jedes Studiengangs
    total_groups = sum(configuration['study_programs'].values())
    relative_sizes = {sg: groups / total_groups for sg, groups in configuration['study_programs'].items()}

    # Berechne die ideale Verteilung unbeliebter Termine basierend auf relativer Größe
    total_unpopular = sum(unpopular_terms.values())

    # Für jeden Studiengang
    for sg_idx, sg_name in enumerate(sg_in_week):
        # Relative Größe dieses Studiengangs
        size_factor = relative_sizes.get(sg_name, 0)

        # Idealwert für unbeliebte Termine für diesen Studiengang
        # basierend auf bisheriger Gesamtzahl und relativer Größe
        ideal_unpopular = total_unpopular * size_factor if total_unpopular > 0 else 0

        # Aktuelle Anzahl unbeliebter Termine
        current_unpopular = unpopular_terms.get(sg_name, 0)

        # Wenn der Studiengang bereits mehr als seinen fairen Anteil hat,
        # begrenzen wir strenger die neuen unbeliebten Termine
        if current_unpopular > ideal_unpopular * 1.1:  # 10% Toleranz
            # Zähle potenzielle neue unbeliebte Termine
            unpopular_count = []

            # Unbeliebte Tage
            for day in unpopular_days:
                for slot_idx in range(1, 5):
                    for room in ['A', 'B']:
                        var_name = f"slot_{slot_idx}_room_{room}_{day}_{week}"
                        if var_name in variables:
                            is_assigned = model.NewBoolVar(f"{var_name}_is_{sg_name}_unpopular_day")
                            model.Add(variables[var_name] == sg_idx).OnlyEnforceIf(is_assigned)
                            model.Add(variables[var_name] != sg_idx).OnlyEnforceIf(is_assigned.Not())
                            unpopular_count.append(is_assigned)

            # Unbeliebte Zeitslots
            for day in days:
                for slot_idx in unpopular_slot_indices:
                    for room in ['A', 'B']:
                        var_name = f"slot_{slot_idx}_room_{room}_{day}_{week}"
                        if var_name in variables:
                            is_assigned = model.NewBoolVar(f"{var_name}_is_{sg_name}_unpopular_slot")
                            model.Add(variables[var_name] == sg_idx).OnlyEnforceIf(is_assigned)
                            model.Add(variables[var_name] != sg_idx).OnlyEnforceIf(is_assigned.Not())
                            unpopular_count.append(is_assigned)

            # Begrenze die Anzahl neuer unbeliebter Termine
            # Je stärker die Abweichung, desto strikter die Begrenzung
            if unpopular_count:
                if current_unpopular > ideal_unpopular * 1.5:
                    # Sehr streng, nur 1 neuer unbeliebter Termin erlaubt
                    max_new_unpopular = 1
                elif current_unpopular > ideal_unpopular * 1.25:
                    # Streng, nur 2 neue unbeliebte Termine erlaubt
                    max_new_unpopular = 2
                else:
                    # Normal, 3 neue unbeliebte Termine erlaubt
                    max_new_unpopular = 3

                model.Add(sum(unpopular_count) <= max_new_unpopular)


def add_week_soft_constraints(model, variables, configuration, sg_in_week, week):
    """
    Fügt die weichen Constraints als Teil der Zielfunktion für eine Woche hinzu.
    """
    # Code bleibt unverändert - siehe vorherige Implementierung
    # ...
    # (Ich lasse diesen Teil aus, da er umfangreich ist und unverändert bleibt)
    # ...

    # Daten aus der Konfiguration extrahieren
    days = list(configuration['days'].keys())

    # Liste der Studiengänge mit mehreren Kursgruppen
    study_programs_with_more_than_one_course = [
        sg for sg in sg_in_week if configuration['study_programs'].get(sg, 0) > 1
    ]

    # Zielfunktionsterme
    objective_terms = []

    # 1. Weicher Constraint: Studiengänge mit mehreren Kursgruppen sollten
    # beide Räume gleichzeitig nutzen können
    parallel_weight = configuration.get('room_popularity', {}).get('parallel', 7)

    for day in days:
        for slot_idx in [1, 2, 3, 4]:
            # Für jeden Tag und Zeitslot
            var_name_a = f"slot_{slot_idx}_room_A_{day}_{week}"
            var_name_b = f"slot_{slot_idx}_room_B_{day}_{week}"

            if var_name_a not in variables or var_name_b not in variables:
                continue

            # Für jeden Studiengang mit mehreren Kursgruppen
            for sg_name in study_programs_with_more_than_one_course:
                if sg_name in sg_in_week:
                    sg_idx = sg_in_week.index(sg_name)

                    # Erstelle eine Binärvariable, die angibt, ob der Studiengang
                    # beide Räume gleichzeitig nutzt
                    uses_both_rooms = model.NewBoolVar(
                        f"sg_{sg_name}_uses_both_rooms_{day}_{slot_idx}_{week}"
                    )

                    # Diese Variable ist 1, wenn der Studiengang beide Räume nutzt
                    model.Add(variables[var_name_a] == sg_idx).OnlyEnforceIf(uses_both_rooms)
                    model.Add(variables[var_name_b] == sg_idx).OnlyEnforceIf(uses_both_rooms)

                    # Füge einen Bonus zur Zielfunktion hinzu
                    objective_terms.append(uses_both_rooms * parallel_weight)

    # 2. Weicher Constraint: Zeitslot-Popularität berücksichtigen
    slot_names = ["Slot A", "Slot B", "Slot C", "Slot D"]

    for var_name, var in variables.items():
        # Extrahiere den Slot-Index aus dem Variablennamen
        parts = var_name.split('_')
        slot_idx = int(parts[1])

        # Entsprechenden Zeitslot und seine Popularität ermitteln
        slot_name = slot_names[slot_idx-1]
        slot_popularity = configuration['time_slots'][slot_name]['popularity']

        # Für jeden Studiengang eine Bewertung basierend auf dem Zeitslot hinzufügen
        for sg_idx in range(len(sg_in_week)):
            is_assigned = model.NewBoolVar(f"{var_name}_is_sg_{sg_idx}")
            model.Add(var == sg_idx).OnlyEnforceIf(is_assigned)

            # Füge einen Bonus basierend auf der Slot-Popularität hinzu
            objective_terms.append(is_assigned * slot_popularity)

    # 3. Weicher Constraint: Wochentag-Popularität berücksichtigen
    for var_name, var in variables.items():
        # Extrahiere den Tag aus dem Variablennamen
        parts = var_name.split('_')
        day_index = parts.index(week) - 1
        day = parts[day_index]

        # Popularität des Tages ermitteln
        day_popularity = configuration['days'][day]['popularity']

        # Für jeden Studiengang eine Bewertung basierend auf dem Tag hinzufügen
        for sg_idx in range(len(sg_in_week)):
            is_assigned = model.NewBoolVar(f"{var_name}_is_day_{sg_idx}")
            model.Add(var == sg_idx).OnlyEnforceIf(is_assigned)

            # Füge einen Bonus basierend auf der Tag-Popularität hinzu
            objective_terms.append(is_assigned * day_popularity)

    # 4. Weicher Constraint: Sequentielle Prüfungen
    sequential_weight = configuration.get('room_popularity', {}).get('sequential', 5)

    for day in days:
        for sg_idx in range(len(sg_in_week)):
            # Prüfe für aufeinanderfolgende Zeitslots
            for slot_idx in range(1, 4):  # Slots 1, 2, 3 (da wir den nachfolgenden betrachten)
                # Variablen für aktuelle und nächste Zeitslots
                current_slots = []
                next_slots = []

                for room in ['A', 'B']:
                    current_var_name = f"slot_{slot_idx}_room_{room}_{day}_{week}"
                    next_var_name = f"slot_{slot_idx+1}_room_{room}_{day}_{week}"

                    if current_var_name not in variables or next_var_name not in variables:
                        continue

                    current_is_sg = model.NewBoolVar(f"{current_var_name}_is_sg_{sg_idx}")
                    next_is_sg = model.NewBoolVar(f"{next_var_name}_is_sg_{sg_idx}")

                    model.Add(variables[current_var_name] == sg_idx).OnlyEnforceIf(current_is_sg)
                    model.Add(variables[next_var_name] == sg_idx).OnlyEnforceIf(next_is_sg)

                    current_slots.append(current_is_sg)
                    next_slots.append(next_is_sg)

                # Für jede Kombination prüfen, ob aufeinanderfolgende Slots
                # vom gleichen Studiengang belegt werden
                for current in current_slots:
                    for next_slot in next_slots:
                        sequential = model.NewBoolVar(
                            f"sg_{sg_idx}_sequential_{day}_{slot_idx}_{week}"
                        )
                        model.AddBoolAnd([current, next_slot]).OnlyEnforceIf(sequential)

                        # Füge einen Bonus zur Zielfunktion hinzu
                        objective_terms.append(sequential * sequential_weight)

    # 5. Weicher Constraint: Vermeidung ungünstiger Abfolgen (spät/früh)
    for day_idx in range(len(days) - 1):
        day = days[day_idx]
        next_day = days[day_idx + 1]

        # Für jeden Studiengang
        for sg_idx in range(len(sg_in_week)):
            # Prüfe, ob der Studiengang einen späten Slot heute hat
            has_late_today = []
            for slot_idx in [3, 4]:  # Späte Slots
                for room in ['A', 'B']:
                    var_name = f"slot_{slot_idx}_room_{room}_{day}_{week}"
                    if var_name in variables:
                        is_late = model.NewBoolVar(f"{var_name}_is_late_for_sg_{sg_idx}")
                        model.Add(variables[var_name] == sg_idx).OnlyEnforceIf(is_late)
                        has_late_today.append(is_late)

            # Prüfe, ob der Studiengang einen frühen Slot morgen hat
            has_early_tomorrow = []
            for slot_idx in [1, 2]:  # Frühe Slots
                for room in ['A', 'B']:
                    var_name = f"slot_{slot_idx}_room_{room}_{next_day}_{week}"
                    if var_name in variables:
                        is_early = model.NewBoolVar(f"{var_name}_is_early_for_sg_{sg_idx}")
                        model.Add(variables[var_name] == sg_idx).OnlyEnforceIf(is_early)
                        has_early_tomorrow.append(is_early)

            # Füge eine Strafe für ungünstige Abfolgen hinzu, wenn relevante Variablen vorhanden sind
            if has_late_today and has_early_tomorrow:
                for late in has_late_today:
                    for early in has_early_tomorrow:
                        bad_sequence = model.NewBoolVar(
                            f"sg_{sg_idx}_bad_sequence_{day}_{next_day}_{week}"
                        )
                        model.AddBoolAnd([late, early]).OnlyEnforceIf(bad_sequence)

                        # Füge eine negative Gewichtung zur Zielfunktion hinzu
                        objective_terms.append(bad_sequence.Not() * 25)  # Hohe Gewichtung

    # 6. Weicher Constraint: Täglicher Slot für jeden Studiengang
    for day in days:
        for sg_idx in range(len(sg_in_week)):
            # Prüfe, ob der Studiengang an diesem Tag mindestens einen Slot hat
            has_slot_today = []

            for slot_idx in [1, 2, 3, 4]:
                for room in ['A', 'B']:
                    var_name = f"slot_{slot_idx}_room_{room}_{day}_{week}"
                    if var_name in variables:
                        is_assigned = model.NewBoolVar(f"{var_name}_is_assigned_to_sg_{sg_idx}")
                        model.Add(variables[var_name] == sg_idx).OnlyEnforceIf(is_assigned)
                        has_slot_today.append(is_assigned)

            # Wenn es relevante Variablen gibt
            if has_slot_today:
                # Erstelle eine Variable, die angibt, ob der Studiengang
                # an diesem Tag mindestens einen Slot hat
                has_any_slot = model.NewBoolVar(f"sg_{sg_idx}_has_slot_{day}_{week}")
                model.AddBoolOr(has_slot_today).OnlyEnforceIf(has_any_slot)
                model.AddBoolAnd([var.Not() for var in has_slot_today]).OnlyEnforceIf(has_any_slot.Not())

                # Füge einen Bonus zur Zielfunktion hinzu
                objective_terms.append(has_any_slot * 15)

    return objective_terms


def solve_week_sequential(configuration, week, unpopular_terms, time_limit=120):
    """
    Löst das Klausurslotverteilungsproblem für eine einzelne Woche im sequentiellen Ansatz.
    
    Args:
        configuration: Dictionary mit der Konfiguration für das Problem
        week: Die zu betrachtende Woche
        unpopular_terms: Dictionary mit Zählung der unbeliebten Termine pro Studiengang
        time_limit: Zeitlimit in Sekunden
        
    Returns:
        Dictionary mit der Lösung oder None, wenn keine Lösung gefunden wurde
    """
    print(f"Verarbeite {week}...")

    # Modell erstellen
    model = cp_model.CpModel()

    # Variablen erstellen
    variables, sg_in_week = create_variables_for_week(model, configuration, week)

    # Harter Constraint: Fairness für diese Woche
    add_week_fairness_constraint(model, variables, configuration, sg_in_week, week, fairness_toleranz=0.3)

    # Harter Constraint: Begrenze die Dominanz eines Studiengangs pro Tag
    add_week_dominance_constraint(model, variables, configuration, sg_in_week, week, max_slots_per_sg_per_day=4)

    # Identifiziere unbeliebte Tage und Slots
    days = list(configuration['days'].keys())
    day_popularities = {day: config['popularity'] for day, config in configuration['days'].items()}
    unpopular_days = sorted(days, key=lambda d: day_popularities[d])[:2]  # 2 unbeliebteste Tage

    slot_names = list(configuration['time_slots'].keys())
    time_slot_popularities = {slot: config['popularity'] for slot, config in configuration['time_slots'].items()}
    unpopular_slots = sorted(slot_names, key=lambda s: time_slot_popularities[s])[:2]  # 2 unbeliebteste Zeitslots

    # Slot-Namen zu Indizes konvertieren
    slot_indices = {
        "Slot A": 1,
        "Slot B": 2,
        "Slot C": 3,
        "Slot D": 4
    }
    unpopular_slot_indices = [slot_indices[slot] for slot in unpopular_slots]

    # Constraints basierend auf bisherigen Zuweisungen hinzufügen
    add_previous_assignments_constraints(model, variables, sg_in_week, week, unpopular_terms, unpopular_days, unpopular_slot_indices)

    # Weiche Constraints als Teil der Zielfunktion
    objective_terms = add_week_soft_constraints(model, variables, configuration, sg_in_week, week)

    # Zielfunktion maximieren
    model.Maximize(sum(objective_terms))

    # Lösung finden
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit
    status = solver.Solve(model)

    # Lösung auswerten
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # Lösung gefunden
        solution = {}
        for var_name, var in variables.items():
            sg_idx = solver.Value(var)
            sg_name = sg_in_week[sg_idx]
            solution[var_name] = sg_name

            # Zähle unbeliebte Termine für diesen Studiengang
            parts = var_name.split('_')
            slot_idx = int(parts[1])
            day = parts[4]

            # Aktualisiere die Zählung der unbeliebten Termine
            if day in unpopular_days or slot_idx in unpopular_slot_indices:
                unpopular_terms[sg_name] = unpopular_terms.get(sg_name, 0) + 1

        print(f"Lösung für {week} gefunden mit Status: {status}")
        print(f"Objektiver Wert: {solver.ObjectiveValue()}")

        return solution
    else:
        print(f"Keine Lösung für {week} gefunden. Status: {status}")
        return None


def solve_week_with_dynamic_constraints(configuration, week, unpopular_terms, time_limit=120):
    """Löst das Problem mit dynamisch angepassten Constraints."""

    # Analysiere die bisherige Verteilung
    total_groups = sum(configuration['study_programs'].values())
    relative_sizes = {sg: groups / total_groups for sg, groups in configuration['study_programs'].items()}

    total_unpopular = sum(unpopular_terms.values())

    # Berechne ideale Verteilung
    ideal_unpopular = {
        sg: total_unpopular * size_factor if total_unpopular > 0 else 0
        for sg, size_factor in relative_sizes.items()
    }

    # Berechne Abweichungen
    deviations = {
        sg: unpopular_terms.get(sg, 0) - ideal
        for sg, ideal in ideal_unpopular.items()
    }

    print(f"\nAbweichungen vor Woche {week}:")
    for sg, dev in sorted(deviations.items(), key=lambda x: x[1], reverse=True):
        if abs(dev) > 1:
            print(f"  {sg}: {dev:+.1f} vom Idealwert")

    # Liste der Toleranzen, vom strengsten zum lockersten
    fairness_tolerances = [0.3, 0.35, 0.4]

    for tolerance in fairness_tolerances:
        print(f"Versuche Lösung für {week} mit Fairness-Toleranz {tolerance}...")

        # Modell erstellen
        model = cp_model.CpModel()

        # Variablen erstellen
        variables, sg_in_week = create_variables_for_week(model, configuration, week)

        # Harter Constraint: Fairness für diese Woche
        add_week_fairness_constraint(model, variables, configuration, sg_in_week, week, fairness_toleranz=tolerance)

        # Harter Constraint: Begrenze die Dominanz eines Studiengangs pro Tag
        add_week_dominance_constraint(model, variables, configuration, sg_in_week, week, max_slots_per_sg_per_day=4)

        # Identifiziere unbeliebte Tage und Slots
        days = list(configuration['days'].keys())
        day_popularities = {day: config['popularity'] for day, config in configuration['days'].items()}

        # Passe die Anzahl der unbeliebten Tage basierend auf der Toleranz an
        if tolerance == 0.3:
            num_unpopular_days = 2
        else:
            num_unpopular_days = 1

        unpopular_days = sorted(days, key=lambda d: day_popularities[d])[:num_unpopular_days]

        # Ähnliches Vorgehen für Slots
        slot_names = list(configuration['time_slots'].keys())
        time_slot_popularities = {slot: config['popularity'] for slot, config in configuration['time_slots'].items()}
        unpopular_slots = sorted(slot_names, key=lambda s: time_slot_popularities[s])[:num_unpopular_days]

        # Slot-Namen zu Indizes konvertieren
        slot_indices = {"Slot A": 1, "Slot B": 2, "Slot C": 3, "Slot D": 4}
        unpopular_slot_indices = [slot_indices[slot] for slot in unpopular_slots]

        # Füge speziell für Studiengänge mit zu vielen unbeliebten Terminen restriktivere Constraints hinzu
        for sg_idx, sg_name in enumerate(sg_in_week):
            if sg_name in deviations and deviations[sg_name] > 3:  # Wenn mehr als 3 über dem Idealwert
                # Zähle potenzielle neue unbeliebte Termine
                unpopular_count = []

                # Unbeliebte Tage und Slots zusammenfassen
                for day in unpopular_days:
                    for slot_idx in range(1, 5):
                        for room in ['A', 'B']:
                            var_name = f"slot_{slot_idx}_room_{room}_{day}_{week}"
                            if var_name in variables:
                                is_assigned = model.NewBoolVar(f"{var_name}_is_{sg_name}_unpopular")
                                model.Add(variables[var_name] == sg_idx).OnlyEnforceIf(is_assigned)
                                model.Add(variables[var_name] != sg_idx).OnlyEnforceIf(is_assigned.Not())
                                unpopular_count.append(is_assigned)

                for day in days:
                    for slot_idx in unpopular_slot_indices:
                        for room in ['A', 'B']:
                            var_name = f"slot_{slot_idx}_room_{room}_{day}_{week}"
                            if var_name in variables:
                                is_assigned = model.NewBoolVar(f"{var_name}_is_{sg_name}_unpopular")
                                model.Add(variables[var_name] == sg_idx).OnlyEnforceIf(is_assigned)
                                model.Add(variables[var_name] != sg_idx).OnlyEnforceIf(is_assigned.Not())
                                unpopular_count.append(is_assigned)

                # Maximal erlaubte neue unbeliebte Termine basierend auf der Abweichung
                max_new = max(1, int(5 - deviations[sg_name]/3))

                if unpopular_count:
                    model.Add(sum(unpopular_count) <= max_new)

        # Constraints basierend auf bisherigen Zuweisungen bei allen Toleranzen
        add_previous_assignments_constraints(model, variables, sg_in_week, week, unpopular_terms,
                                             unpopular_days, unpopular_slot_indices, configuration)

        # Weiche Constraints als Teil der Zielfunktion
        objective_terms = add_week_soft_constraints(model, variables, configuration, sg_in_week, week)

        # Zielfunktion maximieren
        model.Maximize(sum(objective_terms))

        # Lösung finden
        solver = cp_model.CpSolver()
        solver.parameters.max_time_in_seconds = time_limit
        status = solver.Solve(model)

        # Wenn eine Lösung gefunden wurde, zurückgeben
        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            solution = {}
            for var_name, var in variables.items():
                sg_idx = solver.Value(var)
                sg_name = sg_in_week[sg_idx]
                solution[var_name] = sg_name

                # Zähle unbeliebte Termine für diesen Studiengang
                parts = var_name.split('_')
                slot_idx = int(parts[1])
                day = parts[4]

                if day in unpopular_days or slot_idx in unpopular_slot_indices:
                    unpopular_terms[sg_name] = unpopular_terms.get(sg_name, 0) + 1

            print(f"Lösung für {week} gefunden mit Toleranz {tolerance}, Status: {status}")
            print(f"Objektiver Wert: {solver.ObjectiveValue()}")

            return solution

    # Wenn keine Lösung mit allen Toleranzen gefunden wurde
    print(f"Keine Lösung für {week} mit dynamischen Constraints gefunden.")
    return None


def solve_multi_week_sequential(configuration, time_limit_per_week=120):
    """
    Löst das Klausurslotverteilungsproblem sequentiell Woche für Woche.
    """
    weeks = list(configuration['exam_weeks'].keys())
    complete_solution = {}

    # Zähle die Zuweisungen zu unbeliebten Terminen
    unpopular_terms = {sg: 0 for sg in configuration['study_programs']}

    # Löse für jede Woche einzeln
    for week in weeks:
        # Erste Versuch: Mit Standard-Constraints
        week_solution = solve_week_with_dynamic_constraints(configuration, week, unpopular_terms, time_limit_per_week)
 
        complete_solution[week] = week_solution

        # Ausgabe der aktuellen Verteilung unbeliebter Termine
        print("\nAktuelle Verteilung unbeliebter Termine:")
        for sg, count in sorted(unpopular_terms.items(), key=lambda x: x[1], reverse=True):
            if count > 0:
                print(f"  {sg}: {count} unbeliebte Termine")

    return complete_solution

def solve_week_fallback(configuration, week, unpopular_terms, time_limit=120):
    """
    Fallback-Lösung für eine Woche mit lockereren Constraints.
    """
    print(f"Fallback für {week} mit lockereren Constraints...")

    # Modell erstellen
    model = cp_model.CpModel()

    # Variablen erstellen
    variables, sg_in_week = create_variables_for_week(model, configuration, week)

    # Harter Constraint: Fairness für diese Woche mit erhöhter Toleranz
    add_week_fairness_constraint(model, variables, configuration, sg_in_week, week, fairness_toleranz=0.4)  # Toleranz erhöht

    # Harter Constraint: Begrenze die Dominanz eines Studiengangs pro Tag (bei 4 belassen)
    add_week_dominance_constraint(model, variables, configuration, sg_in_week, week, max_slots_per_sg_per_day=4)

    # Identifiziere unbeliebte Tage und Slots
    days = list(configuration['days'].keys())
    day_popularities = {day: config['popularity'] for day, config in configuration['days'].items()}
    unpopular_days = sorted(days, key=lambda d: day_popularities[d])[:1]  # Nur der unbeliebteste Tag

    slot_names = list(configuration['time_slots'].keys())
    time_slot_popularities = {slot: config['popularity'] for slot, config in configuration['time_slots'].items()}
    unpopular_slots = sorted(slot_names, key=lambda s: time_slot_popularities[s])[:1]  # Nur der unbeliebteste Slot

    # Slot-Namen zu Indizes konvertieren
    slot_indices = {
        "Slot A": 1,
        "Slot B": 2,
        "Slot C": 3,
        "Slot D": 4
    }
    unpopular_slot_indices = [slot_indices[slot] for slot in unpopular_slots]

    # Weiche Constraints als Teil der Zielfunktion
    objective_terms = add_week_soft_constraints(model, variables, configuration, sg_in_week, week)

    # Zielfunktion maximieren
    model.Maximize(sum(objective_terms))

    # Lösung finden
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = time_limit
    status = solver.Solve(model)

    # Lösung auswerten
    if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
        # Lösung gefunden
        solution = {}
        for var_name, var in variables.items():
            sg_idx = solver.Value(var)
            sg_name = sg_in_week[sg_idx]
            solution[var_name] = sg_name

            # Zähle unbeliebte Termine für diesen Studiengang
            parts = var_name.split('_')
            slot_idx = int(parts[1])
            day = parts[4]

            # Aktualisiere die Zählung der unbeliebten Termine
            if day in unpopular_days or slot_idx in unpopular_slot_indices:
                unpopular_terms[sg_name] = unpopular_terms.get(sg_name, 0) + 1

        print(f"Fallback-Lösung für {week} gefunden mit Status: {status}")
        print(f"Objektiver Wert: {solver.ObjectiveValue()}")

        return solution
    else:
        print(f"Keine Fallback-Lösung für {week} gefunden. Status: {status}")
        return None


def display_solution(solution, configuration, week):
    """
    Zeigt die gefundene Lösung für eine Woche an.
    """
    if solution is None or week not in solution:
        print(f"Keine Lösung für Woche {week} vorhanden.")
        return

    # Tage und Slots extrahieren
    days = list(configuration['days'].keys())
    slots = ["8:00-10:00", "10:30-12:30", "13:00-15:00", "15:30-17:30"]

    # Lösung anzeigen
    print(f"\nStundenplan für {week}:")
    print("-" * 80)

    # Kopfzeile mit den Tagen
    header = "Zeitslot       | "
    for day in days:
        header += f"{day:^12} | "
    print(header)
    print("-" * 80)

    # Zeilen für jeden Zeitslot
    for slot_idx, slot_time in enumerate(slots, 1):
        row = f"{slot_time} | "

        for day in days:
            room_a_var = f"slot_{slot_idx}_room_A_{day}_{week}"
            room_b_var = f"slot_{slot_idx}_room_B_{day}_{week}"

            room_a = solution[week].get(room_a_var, "-")
            room_b = solution[week].get(room_b_var, "-")

            cell = f"A:{room_a}, B:{room_b}"
            row += f"{cell:^12} | "

        print(row)
    print("-" * 80)


def display_all_solutions(solution, configuration):
    """
    Zeigt die gefundenen Lösungen für alle Wochen an.
    """
    if solution is None:
        print("Keine Lösung vorhanden.")
        return

    # Für jede Woche die Lösung anzeigen
    for week in solution.keys():
        display_solution(solution, configuration, week)

def analyze_and_rebalance_solution(solution, configuration):
    """Analysiert die Gesamtlösung und identifiziert Ungleichgewichte."""

    # Zähle unbeliebte Termine pro Studiengang
    unpopular_counts = {sg: 0 for sg in configuration['study_programs']}

    # Identifiziere unbeliebte Tage und Slots
    days = list(configuration['days'].keys())
    day_popularities = {day: config['popularity'] for day, config in configuration['days'].items()}
    unpopular_days = sorted(days, key=lambda d: day_popularities[d])[:2]

    slot_names = list(configuration['time_slots'].keys())
    time_slot_popularities = {slot: config['popularity'] for slot, config in configuration['time_slots'].items()}
    unpopular_slots = sorted(slot_names, key=lambda s: time_slot_popularities[s])[:2]

    # Slot-Namen zu Indizes konvertieren
    slot_indices = {"Slot A": 1, "Slot B": 2, "Slot C": 3, "Slot D": 4}
    unpopular_slot_indices = [slot_indices[slot] for slot in unpopular_slots]

    # Zähle für jede Woche
    for week, week_solution in solution.items():
        for var_name, sg_name in week_solution.items():
            parts = var_name.split('_')
            slot_idx = int(parts[1])
            day = parts[4]

            if day in unpopular_days or slot_idx in unpopular_slot_indices:
                unpopular_counts[sg_name] = unpopular_counts.get(sg_name, 0) + 1

    # Berechne die ideale proportionale Verteilung
    total_groups = sum(configuration['study_programs'].values())
    total_unpopular = sum(unpopular_counts.values())

    ideal_distribution = {
        sg: (groups / total_groups) * total_unpopular
        for sg, groups in configuration['study_programs'].items()
    }

    # Finde Studiengänge mit zu vielen/wenigen unbeliebten Terminen
    deviations = {
        sg: unpopular_counts.get(sg, 0) - ideal
        for sg, ideal in ideal_distribution.items()
    }

    # Ausgabe der Analyse
    print("\nAnalyse der Gesamtlösung:")
    print("Studiengang | Kursgruppen | Unbeliebte Termine | Ideal | Abweichung")
    print("-" * 70)

    for sg in sorted(configuration['study_programs'].keys()):
        groups = configuration['study_programs'][sg]
        unpopular = unpopular_counts.get(sg, 0)
        ideal = ideal_distribution[sg]
        dev = deviations[sg]

        print(f"{sg:^10} | {groups:^11} | {unpopular:^17} | {ideal:^5.1f} | {dev:^10.1f}")

    # TODO: Hier könnte man einen Algorithmus zur Neuverteilung implementieren
    # für Studiengänge mit zu großen Abweichungen

    return unpopular_counts, ideal_distribution, deviations

def run_multi_week_sequential(configuration):
    """
    Führt das CSP-Modell für die Klausurslotverteilung sequentiell für mehrere Wochen aus.
    """
    # Löse das Problem für alle Wochen sequentiell
    solution = solve_multi_week_sequential(configuration)

    # Zeige die Lösung an
    if solution:
        display_all_solutions(solution, configuration)

    return solution

solution = run_multi_week_sequential(configuration)
print(solution)

### Begründungen 

<...Ihr Text...>

### 3. Test und Bewertung

Wie gut ist die Lösung? Prüfen Sie Ihre Konfiguration (u.a. abhängig von Constraint und Variablenwahl) und testen Sie geeignet. 

In [ ]:
# Ihr Code


### Diskussion der Ergebnisse 

<...Ihr Text...>

# Abschluss

Vergleich der beiden Verfahren. Ggf. ist hier noch Code zum Vergleich zu ergänzen, ansonsten weitgehend durch Text.

<...Ihr Text...>